In [ ]:
# -*- coding: utf-8 -*-
"""Olimp.py - Chatbot especializado em Olimpíadas e Paralimpíadas"""

from pathlib import Path
import requests
from bs4 import BeautifulSoup
import hashlib
import google.generativeai as genai

from google.colab import userdata


def obter_informacoes_olimpiadas():
    """Busca e retorna as notícias sobre as Olimpíadas."""
    url = "https://www.olympics.com/en/news/"  # URL exemplo, substitua pela fonte desejada
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lança uma exceção se houver erro na requisição

        soup = BeautifulSoup(response.content, 'html.parser')
        noticias = []
        for artigo in soup.find_all('article'):
            titulo = artigo.find('h3').text.strip()
            link = artigo.find('a')['href']
            noticias.append((titulo, link))
        return noticias
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter notícias: {e}")
        return []


def processar_pergunta(pergunta):
    """Decide se a pergunta é sobre notícias ou precisa ser enviada ao modelo Gemini."""
    if "notícias" in pergunta.lower():
        return obter_informacoes_olimpiadas()
    else:
        return None


def formatar_resposta(resposta, eh_noticia=False):
    """Formata a resposta do chatbot para uma melhor apresentação."""
    if eh_noticia:
        resposta_formatada = "Aqui estão algumas notícias recentes:\n"
        for titulo, link in resposta:
            resposta_formatada += f"- {titulo} ({link})\n"
    else:
        resposta_formatada = resposta.parts[0].text
    return resposta_formatada


def main():
    """Função principal do programa."""
    # Configurando o modelo
    api_key = userdata.get('SECRET_KEY')
    genai.configure(api_key=api_key)

    generation_config = {
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 0,
    }

    safety_settings = [
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_ONLY_HIGH"
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_ONLY_HIGH"
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_ONLY_HIGH"
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_ONLY_HIGH"
        },
    ]

    system_instruction = "Você é um especialista em Olimpíadas e Paralimpíadas, com amplo conhecimento sobre a história, modalidades, atletas e curiosidades desses eventos."

    model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                                  generation_config=generation_config,
                                  system_instruction=system_instruction,
                                  safety_settings=safety_settings)

    chat = model.start_chat(history=[])

    while True:
        pergunta = input("Faça uma pergunta sobre Olimpíadas ou Paralimpíadas: ")
        if pergunta.lower() == "fim":
            break

        noticias = processar_pergunta(pergunta)
        if noticias:
            resposta_chatbot = formatar_resposta(noticias, eh_noticia=True)
        else:
            resposta = chat.send_message([pergunta])
            resposta_chatbot = formatar_resposta(resposta)

        print(f"Chatbot: {resposta_chatbot}")
        print("-" * 50)


if __name__ == "__main__":
    main()